# Reference: [구글 BERT의 정석](http://www.yes24.com/Product/Goods/104491152)

## <span style='color:gold'>01. 트랜스포머의 등장 이유</span>

- 트랜스포머 이전에 RNN, LSTM 네트워크가 존재했다(다음 단어 예측/next word prediction, 기계번역/machine translation, 텍스트 생성/text generation 등의 순차적 태스크에서 널리 사용되었음).
- 하지만 이러한 네트워크는 <span style='color:red'>장기 의존성 문제/long-term dependency가 있다.</span>
>**[ long-term dependency ]<br>**
RNN이 hidden state를 통해 과거의 정보를 저장할 때 문장의 길이가 길어지면 앞의 과거 정보가 마지막 시점까지 전달되지 못하는 현상.<br>
-> 모델 학습에 있어 과거의 정보를 활용하지 못하기에 긴 문장이 입력됐을 때 성능이 떨어진다.
- 순수하게 어텐션/attention만을 사용한 모델인 트랜스포머는 RNN, LSTM의 한계점을 극복하기 위해 등장했다. (원 논문: [Attention is All You Need](https://arxiv.org/abs/1706.03762))

## <span style='color:gold'>02. 인코더</span>

- 트랜스포머의 인코더는 크게 두 가지로 구성되어있다.
> **멀티 헤드 어텐션(Multi-Head Attention, MHA)**<br>
**피드포워드 네트워크(Feedforward Network)**
- MHA를 이해하기 위해선 셀프 어텐션(self attention)을 이해하고 넘어가야 한다.

### 02-01. 셀프 어텐션

**A dog ate the food because it was hugnry.**<br>
- 다음과 같은 문장이 있을 때, it은 'food'가 아닌 'dog'를 의미한다.
- 사람은 문장 해석을 통해 it이 'dog'를 가리킨다는 것을 알 수 있지만, 모델은 어떻게 알 수 있을까? -> 이 때 self attention을 사용한다.
- 모델은 각 단어의 표현을 순차적으로 학습하는데(e.g. A -> dog -> ate -> ...), self attention을 통해 <span style='color:red'>학습하는 과정에서 문장을 구성하는 다른 모든 단어들과의 관계 또한 배우게 된다.</span>
- 간단하게 정리하자면 self attention은 **단어와 단어 사이의 관계를 파악**하기 위해 사용하는 것이라고 할 수 있다.

**self attention의 동작 원리**
- self attention을 계산하기 위해선 Q(Query), K(Key), V(Value) matrix가 필요하다.
- Q, K, V matrix는 입력 matrix로 부터 생성된다.
- 입력 matrix는 (문장 길이 x 임베딩 차원)의 shape을 갖는 matrix이다.

In [1]:
import torch
import torch.nn as nn

**[입력 matrix 생성]**

In [2]:
input_string = 'I am good'
word_list = input_string.split()
print(word_list)

embedding_dim = 512
input_matrix = torch.randn(len(word_list), embedding_dim) # (단어 길이, 임베딩 차원)의 모양을 갖는 입력 행렬 생성
print(input_matrix.shape)

['I', 'am', 'good']
torch.Size([3, 512])


**[Q, K, V matrix 생성]**

In [3]:
qkv_dim = 64
weight_Q = nn.Linear(embedding_dim, qkv_dim)
weight_K = nn.Linear(embedding_dim, qkv_dim)
weight_V = nn.Linear(embedding_dim, qkv_dim)

Q = weight_Q(input_matrix)
K = weight_K(input_matrix)
V = weight_V(input_matrix)

print(f'query: {Q.shape}')
print(f'key: {K.shape}')
print(f'value: {V.shape}')

query: torch.Size([3, 64])
key: torch.Size([3, 64])
value: torch.Size([3, 64])


**[self attention 계산]**

In [4]:
# 01. Q와 K.t() 간 내적 연산
out = torch.matmul(Q, K.t())
print(out)

tensor([[ 3.2006,  1.3753,  0.6912],
        [-1.7001,  2.9440, -2.4974],
        [-1.1335, -1.5982, -0.7392]], grad_fn=<MmBackward0>)


- Q와 K.t() 간 내적 연산의 의미는 무엇인가?
- Q, K, V 행렬을 구할 때 달라진 것은 임베딩 차원일 뿐, **문장의 길이는 똑같다.** 즉 Q, K, V의 각 행은 'I', 'am', 'good'을 표현하는 벡터들을 의미한다.
- Q와 K.t()간의 내적 연산은, 하나의 단어와 전체 단어간의 내적을 구하는 것이라고 생각할 수 있다.
- 출력 결과(3x3 matrix)를 보면 이해하기 쉬운데, **<span style='color:red'>(0, 0)은 'I'와 'I'의 내적, (0, 1)은 'I'와 'am'의 내적, (0, 2)는 'I'와 'good'의 내적을 의미한다.</span>**
- 내적은 **두 벡터 간 유사성**을 의미한다. 'I'와 'am' 간의 내적은, 두 단어 간 유사성을 구하는 것이라고 설명할 수 있다.
- 내적을 기하학적으로 생각해보면, 하나의 벡터를 또 다른 벡터에 투영(projection)시키는 것이다. 벡터의 크기(magnitude)는 특정 벡터를 나타낼 수 있는 속성 중 하나이기에, 투영시킨 벡터의 크기는 곧 두 벡터의 유사도라고 생각할 수 있다.
- **결국 out은 단어와 단어 간 연관성을 나타내는 matrix라고 할 수 있다.** (지금은 값이 아무 의미를 나타내지 않지만, 학습 과정을 통해 단어 간 관계를 파악할 수 있게 된다.)

- 궁금한 점: 입력 문장 matrix 간의 내적을 구해도 될 텐데, 굳이 Q, K, V matrix를 구하는 이유는 무엇인가?
- 내 생각: '학습을 통한 최적화를 가능하게 만들기 위함'이라고 생각한다. 입력 문장 matrix는, 최적화가 불가능한 고정값이다. 허나 Q, K, V matrix는 각각의 weight matrix를 이용해서 생성했기 때문에, 학습 과정을 통한 최적화가 가능하다. Q, K, V matrix가 어떤 방식으로 최적화될 지는 생각하기 어렵지만, loss function을 최적화하는 방향으로 학습될 것은 분명하고 이는 계속해서 정교해진다는 것을 의미한다.
- 실제 이유: 아직 안 찾아봄

In [5]:
# 02. 학습 안정성을 위해 out을 qkv_dim의 제곱근으로 나누어준다. (특별한 이유는 없고, hyperparameter이다.)
# 값을 scaling하면 그만큼 계산되는 gradient도 작아지기 때문에, 학습 시 안정성을 갖출 수 있다.
# 미분은 순간적인 변화량이다. -> 값이 큰 경우 변화량 또한 크다. -> 큰 값으로 weight를 업데이트하면 당연히 크게 변한다. -> 학습이 불안정해진다. -> 따라서 scaling이 필요하다.
from math import sqrt

print(out.dtype)
print(sqrt(qkv_dim))
out = out / sqrt(qkv_dim)
print(out)

torch.float32
8.0
tensor([[ 0.4001,  0.1719,  0.0864],
        [-0.2125,  0.3680, -0.3122],
        [-0.1417, -0.1998, -0.0924]], grad_fn=<DivBackward0>)


In [6]:
# 03. Softmax() 함수를 이용해서 값을 정규화(0~1 사이의 값으로 만드는 것)한다.
softmax = nn.Softmax(dim=1)
attention_score = softmax(out) # 단어 간 연관성을 나타내는 attention score matrix 연산
print(attention_score)

print(torch.sum(attention_score[0, :])) # 행 방향(dim=1) 총합이 1인지 확인
print(torch.sum(attention_score[:, 0])) # 열 방향(dim=0) 총합이 0이 아닌지 확인

tensor([[0.3958, 0.3150, 0.2892],
        [0.2708, 0.4840, 0.2452],
        [0.3340, 0.3151, 0.3509]], grad_fn=<SoftmaxBackward0>)
tensor(1., grad_fn=<SumBackward0>)
tensor(1.0006, grad_fn=<SumBackward0>)


In [7]:
# 04. attention matrix 연산(주의할 점은, 내적이 아닌 weighted sum!)
# weighted sum이기에 이중 for문 말곤 없는 것 같다.

row, col = attention_score.shape
attention = torch.zeros(V.shape)
for i in range(row):
    for j in range(col):
        attention[i, :] += (attention_score[i][j]*V[i, :]) # score를 하나씩 가져와서, V의 행에 곱한 뒤 attention matrix에 더하기

print(attention.shape)

torch.Size([3, 64])


**[self attention의 의미]**

- V matrix의 각 행은 각 단어를 표현하는 벡터 값이다.
- 거기에 attention score를 곱했다. Attention score는 단어 간 연관성을 나타내는 matrix이다.
- 단어 간 연관성과 단어를 곱했기 때문에, 연관성이 높은 단어일 수록 attention matrix에 더 많은 값이 반영될 것이다.
- 즉 'I'를 나타내는 attention에는 'I'의 V값이 많이 반영될 것이고, 'am'을 나타내는 attention에는 'am'의 V 값이 많이 반영될 것이다.
- 쉽게 설명하자면 **함유량이 다르다고 생각할 수 있다.**
- 물리적인 의미를 생각해봤을 때 attention은 **<span style='color:red'>문장의 문맥 정보를 담고 있다고 해석할 수 있겠다.</span>**

### 02-02 멀티 헤드 어텐션(Multi-Head Attention, MHA)

- self attention은 문장의 문맥 정보를 담고 있다(단어 간에 의미가 얼마나 유사한지)
- 하지만 주의할 점은, **attention의 학습이 올바른 문맥의 학습을 보장하지 않는다는 점이다.**
- 'I am a man.'이라는 문장에서 I는 man과 연관이 있다.
- 해당 문장에 대해 attention을 학습시키면, I와 man의 연관성이 올바르게 학습됐을 수도 있고, 아닐 수도 있다.
- 한 문장의 경우 쉽다고 생각할 수 있겠지만, 실제 모델 학습에 사용되는 데이터는 엄청나게 많다. 모든 데이터에 대해 모델이 학습하는 과정에서 올바른 문맥을 학습할지 아닐지는 증명이 어렵다.
- 그래서 MHA가 등장했다. <span style='color:red'>**잘못된 문맥을 배울 확률을 낮추기 위해서 말이다.**</span> 다르게 말하면 **정확도를 높인다**고 할 수 있다. 일종의 보험이다. 
- Self attention을 여러 개 둔 것이 Multi-Head Attention이다. Attention 각각을 계산한 다음, concatenate(정해진 축에 따라 연결하는 것)한 뒤 weight matrix와 dot-product 해줌으로써 결과값을 얻는다.

In [8]:
# Multi-Head Attention 구현
import torch

# 01. 입력 문장을 벡터로 변환
def sentence_to_vector(sentence: str, embedding_dim: int):
  sentence = sentence.replace('.', '')  # 단순 불용어 제거. 실제로는 다양한 library의 tokenizer 이용
  word_list = sentence.split()          # 공백을 기준으로 분리

  return torch.randn((len(word_list), embedding_dim)) # (문장 길이, 임베딩 차원) shape의 가우시안 분포를 따르는 텐서 반환

input = 'Transformer is awesome.'
embedded_vec = sentence_to_vector(input, 512)
print(f'type: {type(embedded_vec)}, shape: {embedded_vec.shape}')

type: <class 'torch.Tensor'>, shape: torch.Size([3, 512])


In [9]:
# 02. 원하는 개수 만큼의 Q, K, V matrix 생성
import torch
import torch.nn as nn

torch.manual_seed(0)  # Reproducibility(재현)를 위한 시드 설정. PyTorch의 weight들은 random initialization되기 때문에, 재현을 위해 시드 설정이 필요
def qkv_generator(vec: torch.tensor, embedding_dim: int, num: int)-> tuple:
  result = []
  for _ in range(num):
    weight_q = nn.Linear(vec.shape[-1], embedding_dim)
    weight_k = nn.Linear(vec.shape[-1], embedding_dim)
    weight_v = nn.Linear(vec.shape[-1], embedding_dim)

    query = weight_q(vec)
    key = weight_k(vec)
    value = weight_v(vec)

    result.append((query, key, value))
  
  return tuple(result)

qkv_matrices = qkv_generator(embedded_vec, 64, 4)
print(f'Total numbers of qkv matrices: {len(qkv_matrices)}')
for qkv_matrix in qkv_matrices:
  q, k, v = qkv_matrix[0], qkv_matrix[1], qkv_matrix[2]
  print(f'query.shape: {q.shape}, query: {q[0, :3]}')
  print(f'key.shape: {k.shape}, key: {k[0, :3]}')
  print(f'value.shape: {v.shape}, value: {v[0, :3]}')
  print()

Total numbers of qkv matrices: 4
query.shape: torch.Size([3, 64]), query: tensor([-0.4001,  0.3124,  0.0872], grad_fn=<SliceBackward0>)
key.shape: torch.Size([3, 64]), key: tensor([ 0.5620, -0.0283,  0.3955], grad_fn=<SliceBackward0>)
value.shape: torch.Size([3, 64]), value: tensor([0.4760, 0.0479, 0.6244], grad_fn=<SliceBackward0>)

query.shape: torch.Size([3, 64]), query: tensor([-0.3928, -0.1612, -0.1454], grad_fn=<SliceBackward0>)
key.shape: torch.Size([3, 64]), key: tensor([1.3861, 0.8733, 0.5070], grad_fn=<SliceBackward0>)
value.shape: torch.Size([3, 64]), value: tensor([2.0495, 1.5642, 0.1738], grad_fn=<SliceBackward0>)

query.shape: torch.Size([3, 64]), query: tensor([-0.3792,  0.7973, -0.2231], grad_fn=<SliceBackward0>)
key.shape: torch.Size([3, 64]), key: tensor([-0.4805, -0.3987, -0.4330], grad_fn=<SliceBackward0>)
value.shape: torch.Size([3, 64]), value: tensor([ 0.7827, -0.3253,  0.2540], grad_fn=<SliceBackward0>)

query.shape: torch.Size([3, 64]), query: tensor([ 0.1879, 

In [13]:
# 03. MHA 구현
import torch
import torch.nn as nn
from math import sqrt

torch.manual_seed(0)
def multi_head_attention(input: tuple)-> torch.tensor:
  result = []
  softmax = nn.Softmax(dim=1)
  for matrix in input:
    q, k, v = matrix[0], matrix[1], matrix[2]
    attention_score = softmax((torch.matmul(q, k.T)) / sqrt(q.shape[-1]))
    
    row, col = attention_score.shape
    attention_matrix = torch.zeros_like(q)
    for i in range(row):
      for j in range(col):
        attention_matrix[i, :] += (attention_score[i][j] * v[i, :])

    result.append(attention_matrix)
  
  result = torch.cat(tuple(result), dim=-1)  # embedding_dim에 맞추어 concat한다.
  weight_mha = nn.Linear(result.shape[-1], input[0][0].shape[-1])
  result = weight_mha(result)
  print(result.shape)

multi_head_attention(qkv_matrices)

torch.Size([3, 64])


### 02-03 위치 인코딩(Positional Encoding)

- 위치 인코딩은 왜 필요한가? 먼저 RNN과 Transformer의 차이를 알아야 한다.
> RNN: 모델의 입력이 **단어**이다.<br>
Transformer: 모델의 입력이 **문장**이다.

- 예를 들어, "I am good." 이라는 문장이 주어졌다고 하자.
- 우선 입력 문장에서 불용어(모델 성능에 악영향을 끼칠 수 있는, 무의미한 단어들)들을 제거하고, 단어 단위로 분리한다.
- 그 결과로 'I', 'am', 'good'이라는 단어들을 얻을 수 있다.
- 컴퓨터는 문자를 알 수 없다. 따라서 각 단어들을 고유한 벡터 값으로 변환한 뒤 모델에 입력한다.
- 일련의 과정을 통해 v_I, v_am, v_good 이라는 세 개의 벡터값을 얻었다. 지금부터가 RNN과 Transformer의 차이점이다.
---
- RNN은 각 벡터 값들을 순차적으로 학습한다.<br>
  1. v_I를 입력받고, 학습한다.
  1. v_am을 입력받고, 학습한다.
  1. v_good을 입력받고 학습한다.
- <span style='color:red'>**문장의 의미를 파악하는 것에 있어서, 단어들의 순서 정보는 필수적**이다.</span> 20개의 단어로 구성된 문장이 있다고 할 때, 단어의 순서를 모른다면 해당 문장의 의미를 파악하는 것은 불가능에 가깝다.
- RNN은 단어들을 순차적으로 입력받아 학습하기 때문에, 자연스레 순서 정보 또한 학습할 수 있다. 이는 sequential한 모델들의 장점이다.
- 단점은 장기 의존성(long term dependency) 문제가 발생한다는 것이다. Sequential 모델들의 공통적인 단점이기도 하다.
> 장기 의존성 문제: 문장의 길이가 길어질수록 초기 정보를 잃어버리는 문제
- RNN의 학습 방식과 관련이 있는데, 여기서 다루기엔 내용이 너무 길어져서 넘어간다.
---
- Transformer는 단어 벡터들을 병렬 형태로 합쳐 문장 matrix를 만들고, 이를 학습에 이용한다.
- 한 번에 문장 전체를 입력받기 때문에, 장기 의존성 문제에서 훨씬 자유롭다.
- 다만 의미 파악에 중요한 단서인 단어들의 순서 정보가 없다. 이를 해결하기 위해 Positional Encoding을 사용하는 것이다.
---
- 한 가지 의문점: 단어 벡터들을 특정 축에 따라 순서대로 쌓는다면, 이 또한 순서 정보가 아닐까?
- 내 생각: 학습 과정에서 순서 정보를 찾아낼 수도 있겠지만, 명시적으로 순서 정보를 주는 것이 성능 면에서 훨씬 좋을 것 같긴 하다.


- 그렇다면 어떻게 위치 정보를 더해줄 수 있을까?
- 정현파(Sinusoidal) 함수를 이용한다.
> Sinusoidal function: 주기 함수. 사인(sine) 함수를 지칭하는 말이긴 하지만, 다른 주기함수들을 포함하는 말로 사용하기도 한다.
- 행렬 요소들의 위치에 따라 값이 달라지는데, 수식은 다음과 같다.
- pos는 문장 속 단어의 위치(row_index)이고, i는 해당 element의 위치를 나타내기 위해 2i 혹은 2i+1에 들어가는 정수값이다.
> $$ P(pos, 2i) = sin(\frac{pos}{10000^{2i/d_{model}}}) $$
$$ P(pos, 2i+1) = cos(\frac{pos}{10000^{2i/d_{model}}}) $$
- 잠시 쉬겠습니다.


## Transformer 최종 구현

In [11]:
class encoder(nn.Module):
    def __init__():
        super().__init__()
    
    def forward(self, x):
    
class decoder(nn.Module):
    def __init__():
        super().__init__()
        
    def forward(self, x):

IndentationError: ignored

In [ ]:
class Transformer(nn.Module):
    def __init__():
        super().__init__()
        self.encoder = 
        self.decoder = 
        
    def forward(self, x):
        